## Roadmap

1. Define Proof of Concept (POC) design
    1. What classes are we going to use?
    2. What are the requirements?
    3. Finish out sport/league pair mapper
    4. Add basic sport selector to the ORM
2. Build out testing suite and documentation for POC
3. Run POC and verify results
4. Define Publish to Pypi pipeline

## 1. Define Proof of Concept (POC) design

**What classes are we going to use?**
- Base
- Sport
- League
- Teams
- Venues
- Scoreboard

**What are the requirements?**


**Finish out sport/league pair mapper**


**Add basic sport selector to the ORM**

## 2. Create sample data collection pipeline for local 
- For the last 2 seasons lets get all the events
- For NFL lets get all the teams for each season
- For each of these events can we build out aggregate metrics 

In [1]:
pip install -U espn-api-orm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Create SportLeague Class

In [1]:
from espn_api_orm.consts import ESPNSportLeagueTypes, ESPNSportTypes
from espn_api_orm.league.api import ESPNLeagueAPI


class SportLeague:
    def __init__(self, sport_league: ESPNSportLeagueTypes, autofill=False):
        self.sport_league = sport_league
        self.sport = ESPNSportTypes(sport_league.value.split('/')[0])
        self.league_str = sport_league.value.split('/')[1]
        self.api = ESPNLeagueAPI(self.sport, self.league_str)
        self.is_active = self.api.is_active()
        if not self.is_active:
            print(f"""{self.sport.value.upper()}/{self.league_str.upper()} is not active turn autofill to True to still get results.""")
        self.league_obj = None
        self.seasons = None
        self.first_season = None
        self.current_season = None
        self.franchises = None
        self.odds_providers = None
        if self.is_active or autofill:
            self._autofill()
    
    def _autofill(self):
        self.league_obj = self.api.get_league()
        self.seasons = self.api.get_seasons()
        self.first_season = self.api.get_first_season(return_values=True)
        self.current_season = self.api.get_current_season(return_values=True)
        self.franchises = self.api.get_franchises()
        self.odds_providers = self.api.get_odds_providers()
    
    def __str__(self):
        return f"{self.sport.value.upper()}/{self.league_str.upper()} is " + ("active" if self.is_active else "not active")
    
    def __repr__(self):
        return self.__str__()

sport_league = SportLeague(ESPNSportLeagueTypes.FOOTBALL_NFL, autofill=True)
sport_league

FOOTBALL/NFL is not active turn autofill to True to still get results.


FOOTBALL/NFL is not active

## Create SportTeamSeason Class

- This will include the teams schedule for easy linking to events

In [2]:
from espn_api_orm.consts import ESPNSportTypes
from espn_api_orm.team.api import ESPNTeamAPI

class SportTeamSeason:
    def __init__(self, sport: ESPNSportTypes, league: str, season: int, team_id: int, autofill=False):
        self.sport = sport
        self.league = league
        self.season = season
        self.team_id = team_id
        self.api = ESPNTeamAPI(self.sport, self.league, self.season, self.team_id)
        self.team = self.api.get_team()        
        self.event_ids = None
        self.record = None
        self.news = None
        self.depthchart = None
        self.statistics = None

        if autofill:
            self.refresh()

    def __str__(self):
        return f"{self.season} - {self.team.displayName}"
    
    def __repr__(self):
        return self.__str__()

    def _autofill(self):
        self.event_ids = self.api.get_events()
        self.record = self.api.get_record()
        self.news = self.api.get_news()
        self.depthchart = self.api.get_depthchart()
        self.statistics = self.api.get_statistics()

    def refresh(self):
        return self._autofill()
    
class SportTeamEvent:
    def __init__(self, sport: ESPNSportTypes, league: str, season: int, event_id: int, team_id: int, autofill=False):
        self.sport = sport
        self.league = league
        self.season = season
        self.event_id = event_id
        self.team_id = team_id
        

## Create SportEvent Class

The big boy class. We will have to define loaders to be selected for what data will get autofilled vs what we can add after the group has completed

In [3]:
from espn_api_orm.event.api import ESPNEventAPI
from espn_api_orm.event.schema import Event

class SportEvent:
    '''
    Allows for new event collection from the event_id or pass in an existing ESPNEvent object
    '''
    def __init__(self, sport: ESPNSportTypes, league: str, season: int, event_id: int = None, event: Event = None, autofill=False):
        self.sport = sport
        self.league = league
        self.season = season
        self.event = None
        if event:
            self.event_id = event.id
        else:
            self.event_id = event_id
        self.api = ESPNEventAPI(self.sport, self.league, self.event_id)
        self.event = self.api.get_event() if event is None else event
        self.odds = None
        self.officials = None
        self.prediction = None
        if autofill:
            self.refresh()

    def __str__(self):
        return f"{self.season} - {self.event.name}"
    
    def __repr__(self):
        return self.__str__()

    def _autofill(self):
        self.odds = self.api.get_odds()
        self.officials = self.api.get_game_officials()
        self.prediction = self.api.get_prediction()

    def refresh(self):
        return self._autofill()

## Create SportSeason Class


In [8]:
from espn_api_orm.consts import ESPNSportTypes, ESPNSportSeasonTypes
from espn_api_orm.calendar.api import ESPNCalendarAPI
from espn_api_orm.season.api import ESPNSeasonAPI
from espn_api_orm.team.api import ESPNTeamAPI
from espn_api_orm.scoreboard.api import ESPNScoreboardAPI
import datetime
from enum import Enum

## Create a load type enum for sport season
class SportSeasonLoadType(Enum):
    EVENTS = 1
    TEAMS = 2
    BOTH = 3
    NONE = 4

class SportSeason:
    def __init__(self, sport_league: SportLeague, season: int, load_type: SportSeasonLoadType = SportSeasonLoadType.NONE):
        self.sport_league = sport_league
        self.sport = ESPNSportTypes(sport_league.sport)
        self.league_str = sport_league.league_str
        self.season = season
        self.load_type = load_type
        self.api = ESPNSeasonAPI(self.sport, self.league_str, self.season)
        self.types = self.api.get_valid_types()
        self.calendar = self._load_calendar()
        self.teams = None
        self.events = None
        self.refresh()

    def __str__(self):
        return f"{self.sport.value.upper()}/{self.league_str.upper()} - {self.season}"
    
    def __repr__(self):
        return self.__str__()

    def _load_calendar(self):
        calendar_api = ESPNCalendarAPI(self.sport, self.league_str, self.season)
        return calendar_api.get_calendar_sections(self.types)

    def _load_teams(self):
        return [SportTeamSeason(self.sport, self.league_str, self.season, team_id, autofill=True) for team_id in self.api.get_team_ids()[0:3]]

    def _load_events(self):
        ## events for each calendar section
        ## use the scoreboard api to collect the espn sport event
        events = []
        #date_events = {}
        scoreboard_api = ESPNScoreboardAPI(self.sport, self.league_str)
        calendar_dates = []
        ## loop through each calendar section and get unique dates
        for calendar_section in self.calendar:
            calendar_dates.extend(calendar_section.dates)
        
        calendar_dates = list(set(calendar_dates))
        calendar_dates = sorted(calendar_dates)

        
        for calendar_date in calendar_dates[0:2]:
            date = datetime.datetime.strftime(calendar_date, "%Y%m%d")
            scoreboard = scoreboard_api.get_scoreboard(dates = date)
            #date_events[date] = []
            for event in scoreboard.events:
                sport_event = SportEvent(self.sport, self.league_str, self.season, event=event, autofill=True)
                events.append(sport_event)
                #date_events[date].append(sport_event.event_id)
        return events

    def refresh(self):
        if self.load_type == SportSeasonLoadType.BOTH:
            self.teams = self._load_teams()
            self.events = self._load_events()
        elif self.load_type == SportSeasonLoadType.EVENTS:
            self.events = self._load_events()
        elif self.load_type == SportSeasonLoadType.TEAMS:
            self.teams = self._load_teams()
        else:
            self.events = None
            self.teams = None

        
current_season = sport_league.current_season
sport_season = SportSeason(sport_league, current_season-2, load_type=SportSeasonLoadType.EVENTS)

https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?dates=20220804&limit=1000
https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?dates=20220811&limit=1000


{'id': 401437964,
 'uid': 's:20~l:28~e:401437964',
 'date': datetime.datetime(2022, 8, 5, 0, 0, tzinfo=TzInfo(UTC)),
 'name': 'Jacksonville Jaguars at Las Vegas Raiders',
 'shortName': 'JAX VS LV',
 'season': Season(year=2022, type=1, slug='preseason'),
 'competitions': [Competition(id=401437964, uid='s:20~l:28~e:401437964~c:401437964', date=datetime.datetime(2022, 8, 5, 0, 0, tzinfo=TzInfo(UTC)), attendance=17129, type=Type(id='1', abbreviation='STD'), timeValid=True, neutralSite=True, conferenceCompetition=False, playByPlayAvailable=True, recent=False, venue=Venue(ref=None, id=3718, fullName='Tom Benson Hall of Fame Stadium', address=Address(city='Canton', state='OH', zipCode=None), capacity=None, grass=None, indoor=False, images=None), competitors=[Competitor(id=13, uid='s:20~l:28~t:13', type='team', order=0, homeAway='home', winner=True, team=Team(ref=None, id=13, guid=None, uid='s:20~l:28~t:13', alternateIds=None, slug=None, location='Las Vegas', name='Raiders', nickname=None, abb

In [16]:
e = sport_season.events[0].event

def flatten(event):
    dict_event = event.__dict__
    is_postseason = int(event['season']['type']) == 3
    is_finished = int(dict_event['status'].type.id) == 3

    return {
        'id': dict_event['id'],
        'uid': dict_event['uid'],
        'date': dict_event['date'],
        'name': dict_event['name'],
        'shortName': dict_event['shortName'],
        'season': dict_event['season'].year,
        #'competitions': dict_event['competitions'],
        #'links': dict_event['links'],
        'is_finished': 
    }

flatten(e)

{'id': 401437964,
 'uid': 's:20~l:28~e:401437964',
 'date': datetime.datetime(2022, 8, 5, 0, 0, tzinfo=TzInfo(UTC)),
 'name': 'Jacksonville Jaguars at Las Vegas Raiders',
 'shortName': 'JAX VS LV',
 'season': Season(year=2022, type=1, slug='preseason'),
 'status': Status(clock=0.0, displayClock='0:00', period=4, type=Type(id='3', name='STATUS_FINAL', state='post', completed=True, description='Final', detail='Final', shortDetail='Final'))}

In [1]:
# import espn-api-orm and the LeagueAPI and the VenueAPI
import json
import os
from espn_api_orm.league.api import ESPNLeagueAPI
from espn_api_orm.venue.api import ESPNVenueAPI
from espn_api_orm.consts import ESPNSportLeagueTypes
import re
from geopy.geocoders import Nominatim, ArcGIS
import h3
import time
import requests
import datetime

def get_json_file(path):
    try:
        with open(path, 'r') as file:
            return json.load(file)
    except Exception as e:
        return {}

# Function to save sport league file
def put_json_file(path, data):
    with open(path, 'w') as file:
        json.dump(data, file, indent=4)


def make_geocoding_address_string(name, address):
    items = [name, address.city, address.state, address.zipCode]
    items = list(filter(None, items))
    geocoding_address = ', '.join(items)
    return geocoding_address

def get_elevations(lats, lons, n=50):
    if n > 50:
        raise ValueError("Batch size cannot be greater than 50.")
    try:
        datasets = ','.join([
            # 'aster30m',
            'etopo1',
            # 'eudem25m',
            # 'mapzen',
            'ned10m',
            'srtm30m',
            # 'gebco2020',
            'emod2018',
        ])

        elevations = []
        for i in range(0, len(lats), n):
            batch_lats = lats[i:i + n]
            batch_lons = lons[i:i + n]
            locations = '|'.join([f'{lat},{lon}' for lat, lon in zip(batch_lats, batch_lons)])
            #time.sleep(1.0) # free api
            res = requests.get(f'https://api.opentopodata.org/v1/{datasets}?locations={locations}')
            if res.status_code != 200:
                raise Exception(res.json())
            data = json.loads(res.text)

            batch_elevations = [dataset_response['elevation'] * 3.28084 if dataset_response['elevation'] is not None else None for dataset_response in data['results']]
            elevations.extend(batch_elevations)

        return elevations
    except Exception as e:
        print(e)
        return None

def get_geocoding(name, address):
    if address is None:
        city = None
        state = None
        zip_code = None
    else:
        city = address.city
        state = address.state
        zip_code = address.zipCode
    items = [name, city, state, zip_code]
    items = list(filter(None, items))
    geocoding_address = ', '.join(items)

    geocoding_id = None

    geocoding = {
        'requestedAddress': geocoding_address,
        'codedAddress': None,
        'latitude': None,
        'longitude': None,
        'provider': None,
        'h3Index': None,
        'name': name,
        'city': city,
        'state': state,
        'zipCode': zip_code,
        'elevation': None,
        'lastUpdated': None
    }

    geolocators = {
        "ArcGIS": ArcGIS(), 
        "OSM":Nominatim(user_agent="venue_locator")
    }
    for provider, geolocator in geolocators.items():
        try:
            location = geolocator.geocode(geocoding_address)
            if location:
                geocoding_id = name_filter(location.address)
                geocoding['provider'] = provider
                geocoding['latitude'] = location.latitude
                geocoding['longitude'] = location.longitude
                geocoding['codedAddress'] = location.address
                geocoding['h3Index'] = h3.geo_to_h3(location.latitude, location.longitude, 15)
                geocoding['elevation'] = get_elevations([location.latitude], [location.longitude])[0]
                geocoding['lastUpdated'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                return {geocoding_id: geocoding}
        except:
            continue
    return {geocoding_id: geocoding}

def clean_string(s):
    if isinstance(s, str):
        return re.sub("[^A-Za-z0-9 ]+", '', s)
    else:
        return s


def re_braces(s):
    if isinstance(s, str):
        return re.sub("[\(\[].*?[\)\]]", "", s)
    else:
        return s


def name_filter(s):
    if isinstance(s, str):
        # Adds space to words that
        s = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', s)
        if 'Mary' not in s and ' State' not in s:
            s = s.replace(' St', ' State')
        if 'University' not in s:
            s = s.replace('Univ', 'University')
        if 'zz' in s or 'zzz' in s or 'zzzz' in s:
            s = s.replace('zzzz', '').replace('zzz', '').replace('zz', '')
        s = clean_string(s)
        s = re_braces(s)
        s = str(s)
        s = s.replace(' ', '').lower()
        return s
    else:
        return s

def get_new_venue_coding(sport_str, league_str, venue_id):
    venue = None
    try:
        venue_api = ESPNVenueAPI(sport_str, league_str, venue_id)
        venue = venue_api.get_venue()
        geocoding = get_geocoding(venue.fullName, venue.address)

        if geocoding.keys() == {None}:
            print(f"Geocoding not found for: {sport_str} {league_str} {venue_id}")
        obj = json.loads(venue.json())
        obj.pop('id')
        obj.pop('address')
        new_venue_obj = {
            venue_id : obj
        }

        new_venue_obj[venue_id]['geocodingId'] = list(geocoding.keys())[0]
        new_venue_obj[venue_id]['lastUpdated'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        return new_venue_obj, geocoding
    except Exception as e:
        print(venue)
        raise e

In [5]:
root_path = '../../data'
geocoding_path = root_path+'/' + 'geocoding.json'
geocoding_addresses = get_json_file(geocoding_path)

sport_league_pairs = list(ESPNSportLeagueTypes)
sport_league_pairs = [
    ESPNSportLeagueTypes.FOOTBALL_NFL,
    ESPNSportLeagueTypes.FOOTBALL_COLLEGE_FOOTBALL,
    ESPNSportLeagueTypes.BASKETBALL_MENS_COLLEGE_BASKETBALL,
    ESPNSportLeagueTypes.BASKETBALL_NBA,
    ESPNSportLeagueTypes.BASKETBALL_WNBA,
    ESPNSportLeagueTypes.BASKETBALL_WOMENS_COLLEGE_BASKETBALL,
]
for sport_league in sport_league_pairs:
    sport_str, league_str = sport_league.value.split('/')
    path = f'{root_path}/{sport_str}/{league_str}/'
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)
    venue_path = path + 'venues.json'
    
    # For each LeagueAPI get the venue IDs
    league_api = ESPNLeagueAPI(sport_str, league_str)
    venue_ids = list(map(str,league_api.get_venues()))

    # Try to get the saved sport league venue file if it exists.
    sport_league_venues = get_json_file(venue_path)

    # Handle upsert logic for new or stale venue IDs
    existing_venue_ids = list(sport_league_venues.keys())
    new_venue_ids = list(set(venue_ids) - set(existing_venue_ids))
    new_venue_ids = sorted(new_venue_ids)

    # Skip if no new venues
    if len(new_venue_ids) == 0:
        print(f"No new venues for: {league_str} {sport_str} ")
        continue

    print(f"Getting {len(new_venue_ids)} new venues for: {league_str} {sport_str}")
    new_venues = {}
    new_geocodings = {}
    for venue_id in new_venue_ids:
        new_venue_obj, geocoding = get_new_venue_coding(sport_str, league_str, venue_id)
        new_venues.update(new_venue_obj)
        new_geocodings.update(geocoding)

    print(f"Saving new venues for: {league_str} {sport_str} ...")
    put_json_file(venue_path, {**sport_league_venues, **new_venues})

    # Update geocoding addresses master list
    geocoding_addresses = {**geocoding_addresses, **new_geocodings}
    print(f"Saving new geocodings for: {league_str} {sport_str} ...")
    put_json_file(geocoding_path, geocoding_addresses)
    print("\n")


No new venues for: nfl football 
No new venues for: college-football football 
No new venues for: mens-college-basketball basketball 
Getting 636 new venues for: nba basketball
Saving new venues for: nba basketball ...
Saving new geocodings for: nba basketball ...


Getting 212 new venues for: wnba basketball
Saving new venues for: wnba basketball ...
Saving new geocodings for: wnba basketball ...


Getting 916 new venues for: womens-college-basketball basketball
Saving new venues for: womens-college-basketball basketball ...
Saving new geocodings for: womens-college-basketball basketball ...




In [40]:
import json
import os

from espn_api_orm.league.api import ESPNLeagueAPI
from espn_api_orm.season.api import ESPNSeasonAPI
from espn_api_orm.team.api import ESPNTeamAPI
from espn_api_orm.consts import ESPNSportLeagueTypes
import re
import json

## Keys
# franchiseId - team.id 
# teamId - team.slug
# teamSeasonId - season-team.id

def clean_string(s):
    if isinstance(s, str):
        return re.sub("[^A-Za-z0-9 ]+", '', s)
    else:
        return s


def re_braces(s):
    if isinstance(s, str):
        return re.sub("[\(\[].*?[\)\]]", "", s)
    else:
        return s


def name_filter(s):
    if isinstance(s, str):
        # Adds space to words that
        s = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', s)
        if 'Mary' not in s and ' State' not in s:
            s = s.replace(' St', ' State')
        if 'University' not in s:
            s = s.replace('Univ', 'University')
        if 'zz' in s or 'zzz' in s or 'zzzz' in s:
            s = s.replace('zzzz', '').replace('zzz', '').replace('zz', '')
        s = clean_string(s)
        s = re_braces(s)
        s = str(s)
        s = s.replace(' ', '').lower()
        return s
    else:
        return s

def get_json_file(path):
    try:
        with open(path, 'r') as file:
            return json.load(file)
    except Exception as e:
        return {}

# Function to save sport league file
def put_json_file(path, data):
    with open(path, 'w') as file:
        json.dump(data, file, indent=4)
root_path = '../../data'

sport_league_pairs = list(ESPNSportLeagueTypes)
sport_league_pairs = [
    ESPNSportLeagueTypes.FOOTBALL_NFL,
    ESPNSportLeagueTypes.FOOTBALL_COLLEGE_FOOTBALL
]

def make_team(team):
    id = team.slug
    print('Making Team: ',id )
    return {
        'id': id,
        'franchiseId': team.id,
        'guid': team.guid,
        'uid': team.uid,
        'alternateIds': team.alternateIds,
        'location': team.location.strip() if team.location else None,
        'name': team.name.strip() if team.name else None,
        'nickname': team.nickname.strip() if team.nickname else None,
        'abbreviation': team.abbreviation.strip() if team.abbreviation else None,
        'displayName': team.displayName.strip() if team.displayName else None,
        'shortDisplayName': team.shortDisplayName.strip() if team.shortDisplayName else None,
        'color': team.color.strip() if team.color else None,
        'alternateColor': team.alternateColor.strip() if team.alternateColor else None,
        'isActive': team.isActive,
        'isAllStar': team.isAllStar,
        'college': team.college.strip() if team.college else None,
        'venueId': team.venue.id if team.venue else None,
    }

def make_season_team(season_team, season):
    return {
        'id': f"{season}-{season_team.id}",
        'teamId': season_team.slug,
        'franchiseId': season_team.id,
        'season': season
    }

for sport_league in sport_league_pairs:
    sport_str, league_str = sport_league.value.split('/')
    path = f'{root_path}/{sport_str}/{league_str}/'
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

    # Get existing teams for sport league
    teams_path = path + 'teams.json'
    existing_teams = get_json_file(teams_path)
    # convert dictionary to list with id key added to team payload
    existing_teams = [{**existing_teams[id], 'id': id} for id in existing_teams]

    league_api = ESPNLeagueAPI(sport_str, league_str)

    # Get new teams
    existing_franchise_ids = [i['franchiseId'] for i in existing_teams] if existing_teams != {} else []
    franchise_ids = league_api.get_franchises()
    new_franchise_ids = list(set(franchise_ids) - set(existing_franchise_ids))
    new_franchise_ids = sorted(new_franchise_ids)

    print(f"Getting {len(new_franchise_ids)} new franchises for: {league_str} {sport_str}")

    teams = league_api.get_teams(new_franchise_ids)
    teams = [make_team(i) for i in teams]
    if existing_teams != {}:
        for existing_team in existing_teams:
            if existing_team['franchiseId'] not in new_franchise_ids:
                teams.append(existing_team)
    # Create an id to franchiseId dict mapper to define all teams in league
    team_dict = {i['id']:i['franchiseId'] for i in teams}

    # Get latest season
    seasons = sorted([i for i in league_api.get_seasons() if i > 2001])
    for season in sorted(seasons, reverse=True):
        team_season_path = f"{path}{season}/teams.json"
        season_teams = get_json_file(team_season_path)
        if season_teams != {}:
            seasons = list(range(season, seasons[-1]+1))
            break

    print(f"Running for seasons: {seasons[0]}-{seasons[-1]}")
    # get teams for seasons
    for season in seasons:
        print("*"*20)
        print(f"Season: {season}")
        print("*"*20)
        print()
        # define the season api
        season_api = ESPNSeasonAPI(sport_str, league_str, season)
        if not os.path.exists(path+str(season)):
            os.makedirs(path+str(season), exist_ok=True)

        # Get existing teams for season for the sport league
        team_season_path = f"{path}{season}/teams.json"
        existing_season_teams = get_json_file(team_season_path)
        new_season_teams = []
        existing_season_team_ids= [i['teamId'] for i in existing_season_teams] if existing_season_teams != {} else []

        # Get new teams for season
        season_team_ids = season_api.get_team_season_ids()
        new_season_team_ids = list(set(season_team_ids) - set(existing_season_team_ids))
        new_season_team_ids = sorted(season_team_ids)

        print(f"Getting {len(new_season_team_ids)} new season teams for: {league_str} {sport_str} {season}")

        for team_id in new_season_team_ids:
            # define team api
            team_api = ESPNTeamAPI(sport_str, league_str, season, team_id)
            team_obj = team_api.get_team()
            new_season_team = make_season_team(team_obj, season)
            new_season_teams.append(new_season_team)

            # If team doesn't exist, add it to the teams list and add it to the team dict
            if team_obj.slug not in team_dict:
                teams.append(make_team(team_obj))
                team_dict[team_obj.slug] = team_id

        # Write new season teams to file
        put_json_file(team_season_path, new_season_teams)

    # Write teams to file
    dict_teams = {i.pop('id'):i for i in teams}
    put_json_file(teams_path, dict_teams)

Getting 0 new franchises for: nfl football
Running for seasons: 2024-2024
********************
Season: 2024
********************

Getting 0 new franchises for: college-football football
Running for seasons: 2006-2024
********************
Season: 2006
********************

********************
Season: 2007
********************

Making Team:  amherst-college-lord-jeffs
Making Team:  bridgewater-state-bears
Making Team:  colby-white-mules
Making Team:  curry-colonels
Making Team:  elmhurst-blue-jays
Making Team:  tufts-jumbos
Making Team:  st-olaf-college-oles
Making Team:  washington-mo-bears
Making Team:  hamline-fighting-pipers
Making Team:  southeastern-university-fire
Making Team:  lawrence-vikings
Making Team:  macmurray-college-highlanders
Making Team:  muskingum-muskies
Making Team:  wesleyan-cardinals
Making Team:  kenyon-lords
Making Team:  utica-college-pioneers
Making Team:  wheaton-college-ill-thunder
Making Team:  wilkes-colonels
Making Team:  maranatha-baptist-college-crusa

In [76]:
league_str = 'nfl'

path = f'{root_path}/{sport_str}/{league_str}/'
teams_path = path + 'teams.json'
venue_path = path + 'venues.json'
geocoding_path = root_path+'/' + 'geocoding.json'


teams = get_json_file(teams_path)
geocoding_addresses = get_json_file(geocoding_path)
sport_league_venues = get_json_file(venue_path)

season_teams = get_json_file(path + str(season) + '/teams.json')


In [77]:
full_teams = []
for season_team in season_teams:
    season_team['team'] = teams.get(str(season_team['teamId']), None)
    season_team['venue'] = sport_league_venues.get(str(season_team['team']['venueId']), None)
    season_team['geocoding'] = None
    if season_team['venue'] is not None:
        season_team['geocoding'] = geocoding_addresses.get(season_team['venue']['geocodingId'], None)
    full_teams.append(season_team)

full_teams

[{'id': '2024-1',
  'teamId': 'atlanta-falcons',
  'franchiseId': 1,
  'season': 2024,
  'team': {'franchiseId': 1,
   'guid': '49fd392a-86fe-4df3-1b77-9bbfa18b2ad5',
   'uid': 's:20~l:28~t:1',
   'alternateIds': {'sdr': '8802'},
   'location': 'Atlanta',
   'name': 'Falcons',
   'nickname': 'Falcons',
   'abbreviation': 'ATL',
   'displayName': 'Atlanta Falcons',
   'shortDisplayName': 'Falcons',
   'color': '000000',
   'alternateColor': '000000',
   'isActive': True,
   'isAllStar': False,
   'college': None,
   'venueId': 3495},
  'venue': {'ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/venues/3495?lang=en&region=us',
   'fullName': 'Georgia Dome',
   'capacity': None,
   'grass': False,
   'indoor': True,
   'images': [{'href': 'https://a.espncdn.com/i/venues/nfl/day/3495.jpg',
     'width': 2000,
     'height': 1125,
     'alt': '',
     'rel': ['full', 'day'],
     'lastUpdated': None},
    {'href': 'https://a.espncdn.com/i/venues/nfl/day/interior/3495.jp

In [4]:
from espn_api_orm.consts import ESPNSportLeagueTypes
for i in ESPNSportLeagueTypes:
    print(i.value)

baseball/college-baseball
baseball/college-softball
baseball/mlb
basketball/mens-college-basketball
basketball/nba
basketball/wnba
basketball/womens-college-basketball
field-hockey/womens-college-field-hockey
football/college-football
football/nfl
golf/liv
golf/lpga
golf/pga
hockey/mens-college-hockey
hockey/nhl
hockey/womens-college-hockey
lacrosse/mens-college-lacrosse
lacrosse/pll
lacrosse/womens-college-lacrosse
mma/ufc
soccer/uefa.champions
soccer/eng.1
soccer/esp.1
soccer/ger.1
soccer/usa.1
soccer/ita.1
soccer/fra.1
soccer/eng.2
soccer/ned.1
soccer/por.1
volleyball/mens-college-volleyball
volleyball/womens-college-volleyball
water-polo/mens-college-water-polo
water-polo/womens-college-water-polo
